In [128]:
%matplotlib nbagg
import pandas as pd
import os, sys
import numpy as np
from collections import defaultdict
from caseDetection import detect_case
#import matplotlib as mpl
#mpl.use("GTK3cairo")
import matplotlib.pyplot as plt
#plt.plot([1,2,3,4],'*-')

# DATA ANALYSIS
- per country num obs
- per country num unique sIPs tested

#### reset index, as index were repeated due to concatenation earlier
#### this will not be a problem with any new dataset

In [60]:
df_all = pd.read_pickle("data/detected_case_all_Apr12.pkl").reset_index()

In [61]:
df_sIP_subcat = pd.read_csv("data/Servers_IMC.txt").set_index('sIP')
#df_sIP_subcat

In [162]:
SAMPLENAME = 'sample1'
RESULTS = "results/" + SAMPLENAME + "/"
if not os.path.exists(RESULTS):
    os.makedirs(RESULTS)

### SANITIZE (UNBIAS)
- get only countries with more than 400 unique sIPs
- select only one measurement for each unique sIPs in all countries

#### UNBIASED: so choose only good measurements...

In [62]:
df_good = df_all [  df_all['case'].isin([1,2,3])  ]

In [63]:
# meaningful num of countries with at least 400 unique SIPs
THRESHOLD = 400

# per country, how many unique sIPs
df_unique_sIP_per_country = df_good.groupby(['country'])['sIP'].unique().apply(lambda x:len(x))
df_current = df_unique_sIP_per_country[df_unique_sIP_per_country > THRESHOLD]

In [64]:
valid_countries = df_current.index
TOTAL_NUM_VALID_COUNTRIES = len(valid_countries)
print valid_countries, TOTAL_NUM_VALID_COUNTRIES

# filter df_good to only valid countries
# remove extra columns
df_valid = df_good[  df_good['country'].isin(valid_countries)   ] [['sIP', 'gIP', 'country', 'diff_list', 'domain', 'subcat', 'case']]

Index([u'BR', u'CA', u'CN', u'CR', u'CZ', u'DE', u'DK', u'ES', u'FR', u'GB', u'IN', u'IT', u'JP', u'KR', u'MX', u'MY', u'NL', u'NO', u'PH', u'PL', u'RU', u'SE', u'UA', u'US'], dtype='object') 24


### Unique sIPs per country
- use good cases only (no case 0, -1, 4)
- select only sIPs which measure to all countries TOTAL_NUM_VALID_COUNTRIES
- get their subcats
- get only one measurement per subcat

In [65]:
df_num_unique_countries_per_sIP = df_valid.groupby('sIP')['country'].unique().apply(lambda x: len(x))
df_num_unique_countries_per_sIP.sort(inplace=True)

# SELECT ONLY sIPs WHICH MEASURE TO ALL COUNTRIES
df_sIPs_all_countries = pd.DataFrame(df_num_unique_countries_per_sIP[df_num_unique_countries_per_sIP==TOTAL_NUM_VALID_COUNTRIES])
df_sIP_subcat_represent = df_sIPs_all_countries.join(df_sIP_subcat)

# THE UNBIASED MEASUREMENT
df_sIP_subcat_represent.groupby('subcat')['country'].count()

subcat
adult             16
arts              11
business          11
circum             7
computers          7
games              6
health            11
home               6
kids_and_teens     6
news               4
recreation        10
reference         14
regional           8
science           14
shopping           8
society           13
sports             8
vpn                9
world             11
Name: country, dtype: int64

In [66]:
good_sIPs = df_sIP_subcat_represent.index
print "Unbiased sIPs represented in all good measurements to all countries: ", len(good_sIPs)

Unbiased sIPs represented in all good measurements to all countries:  180


<h3>This data is unbiased because we remove bad cases and keep equal representation of sIPs for every country</h3>
- for these sIPs take a random measurement from each sIP, country group in df_good now

In [67]:
df_good_sIP = df_valid[df_valid['sIP'].isin(good_sIPs)]
gp_good_sIP_country = df_good_sIP.groupby(['sIP','country'])

# even though there are multiple cases per sIP, country pair
# select only one
df_per_sIP_country = gp_good_sIP_country['case'].unique().reset_index()
df_per_sIP_country['count'] = gp_good_sIP_country['case'].count().reset_index()['case']
df_per_sIP_country.head()

,sIP,country,case,count
0,1.209.122.23,BR,"[2, 1]",20
1,1.209.122.23,CA,[2],8
2,1.209.122.23,CN,[2],28
3,1.209.122.23,CR,[2],1
4,1.209.122.23,CZ,[2],2


#### choose one random index of each sip,country pair

In [70]:
# for every sIP, country pair, select a random index of measurement
useful_indices = defaultdict(int)
sanity_check = defaultdict(int)
    
for sIP_country, indices in gp_good_sIP_country.groups.items():
    useful_indices[sIP_country] = np.random.choice( indices )
    sanity_check[sIP_country]+=1
    
print len(useful_indices)
print {k:v for k,v in sanity_check.items() if v>1}

4320
{}


In [71]:
test_indices = useful_indices.values()[:30]

# UNBIASED DATA
- filter df_good_sIP to only the random indices selected
- code under this can be run on current measurements as every sIP, country has just 1 measurement

In [72]:
df_unbiased = df_good_sIP.ix[ useful_indices.values() ]
df_unbiased.to_pickle(RESULTS+"sampled_unbiased_CenP.pkl")
df_unbiased.head()

,sIP,gIP,country,diff_list,domain,subcat,case
14531,74.115.2.240,61.155.176.10,CN,"[3, 2, 1, 4, 1, None, None, 1, 0, 1, 3, 0, 2, ...",hsselite.com,vpn,3
77018,23.59.189.89,80.71.68.80,DK,"[5, 2, 2, 4, 6, 2, 2, 3, 2, 4, 2, 2, 3, 6, 4, ...",news.com.au,news,2
119742,74.117.180.88,78.156.128.125,CZ,"[8, 0, 1, 30, 17, 4, 0, 0, 2, 3, 28, 2, 6, 28,...",videosexarchive.com,adult,2
206627,74.115.2.240,130.206.211.70,ES,"[None, 5, 5, None, None, 4, 3, 3, 1, 3, 1, 2, ...",hsselite.com,vpn,2
145595,208.94.153.100,203.200.163.130,IN,"[None, 1, 1, 0, 0, 1, 0, 0, 1, 3, 0, 0, 0, 0, ...",southwest.com,recreation,2


In [79]:
# each country, sIP has just one entry in df_unbiased
len(df_unbiased)

4320

In [78]:
df_temp = df_unbiased.groupby(['sIP', 'country'])['case'].apply(lambda x: len(x))
df_temp[df_temp > 1]

Series([], name: case, dtype: int64)

In [80]:
grouped_by_country_subcat_case = df_unbiased.groupby(['country', 'subcat', 'case'])
count_per_country_subcat_case = grouped_by_country_subcat_case.count()
count_per_country_subcat_case.head(10)

sIP  gIP  diff_list  domain
country subcat    case                             
BR      adult     1       1    1          1       1
                  2      12   12         12      12
                  3       3    3          3       3
        arts      2      10   10         10      10
                  3       1    1          1       1
        business  1       1    1          1       1
                  2      10   10         10      10
        circum    2       7    7          7       7
        computers 2       5    5          5       5
                  3       2    2          2       2

In [81]:
# replace nan with 0
df_case_count = count_per_country_subcat_case.unstack(level=-1)['sIP'].fillna(0)
df_case_count['total'] = df_case_count.sum(axis=1)
df_case_count.to_html(RESULTS+'cases_per_country_subcat-unbiased-CenP.htm')
df_case_count.head()

case               1   2  3  total
country subcat                    
BR      adult      1  12  3     16
        arts       0  10  1     11
        business   1  10  0     11
        circum     0   7  0      7
        computers  0   5  2      7

#### Censorship by country for each subcat
- create num(subcat) bar plots comparing case2 with (case1 + case3)

In [83]:
# clear all except df_case_count
#del df, df_bad, df_case_default, df_current, df_good, df_merged_unique_cases, df_obs_per_country, df_redo
#del df_sIP_CO, df_temp, df_unique, df_unique_sIP_per_country

In [84]:
group_by_subcat = df_case_count.reset_index().groupby('subcat')

### Per Subcategory Analysis, Excluding error cases

In [85]:
def get_ratios(subcat, indices, df_case_count):
    #if subcat == 'all':
        
    temp_df = df_case_count.ix[ group_by_subcat.groups[subcat] ].reset_index()
    temp_df['censored'] = temp_df[1]+temp_df[3]
    #temp_df['unknown'] = temp_df[-1]+temp_df[0]+temp_df[4]
    #temp_df = temp_df.rename(columns={2:'uncensored'})[['country', 'censored', 'uncensored', 'unknown']].set_index('country')
    temp_df = temp_df.rename(columns={2:'uncensored'})[['country', 'censored', 'uncensored']].set_index('country')

    # calculate ratios
    temp_df['total'] = temp_df.sum(axis=1)
    temp_df['ratio-censored'] = temp_df['censored']/(temp_df['total'])
    temp_df['ratio-uncensored'] = temp_df['uncensored']/(temp_df['total'])
    #temp_df['ratio-unknown'] = temp_df['unknown']/(temp_df['total'])

    # replace no entries with 0
    #df_country_case = temp_df.fillna(0)
    return temp_df

In [86]:
country_case_per_subcat = defaultdict(int)

# iterate over groupby object, indices
for subcat, indices in group_by_subcat.groups.iteritems():
    country_case_per_subcat[subcat] = get_ratios(subcat, indices, df_case_count)
    print subcat, len(country_case_per_subcat[subcat])
    
df_country_case_subcat = pd.concat(country_case_per_subcat)

science 24
arts 24
regional 24
shopping 24
reference 24
business 24
world 24
kids_and_teens 24
computers 24
recreation 24
sports 24
society 24
games 24
adult 24
home 24
health 24
news 24
vpn 24
circum 24


In [163]:
df_country_case_subcat.head(10)

case           censored  uncensored  total  ratio-censored  ratio-uncensored
      country                                                               
adult BR              4          12     16          0.2500            0.7500
      CA              5          11     16          0.3125            0.6875
      CN              3          13     16          0.1875            0.8125
      CR              3          13     16          0.1875            0.8125
      CZ              5          11     16          0.3125            0.6875
      DE              5          11     16          0.3125            0.6875
      DK              4          12     16          0.2500            0.7500
      ES              3          13     16          0.1875            0.8125
      FR              5          11     16          0.3125            0.6875
      GB              5          11     16          0.3125            0.6875

In [88]:
all_results = list(df_country_case_subcat.columns)
print all_results

['censored', 'uncensored', 'total', 'ratio-censored', 'ratio-uncensored']


In [89]:
censorship_ratio = defaultdict(int)
for key in all_results:
    censorship_ratio[key] = df_country_case_subcat[key].unstack(0)
    censorship_ratio[key].to_html(RESULTS+'censorship_'+key+'.html')

In [90]:
censorship_ratio['ratio-censored']

,adult,arts,business,circum,computers,games,health,home,kids_and_teens,news,recreation,reference,regional,science,shopping,society,sports,vpn,world
country,,,,,,,,,,,,,,,,,,,
BR,0.2500,0.090909,0.090909,0.000000,0.285714,0.500000,0.363636,0.000000,0.000000,0.25,0.2,0.000000,0.125,0.142857,0.000,0.307692,0.125,0.222222,0.090909
CA,0.3125,0.090909,0.090909,0.000000,0.285714,0.500000,0.272727,0.333333,0.000000,0.50,0.0,0.000000,0.250,0.214286,0.125,0.230769,0.125,0.444444,0.090909
CN,0.1875,0.090909,0.090909,0.000000,0.571429,0.333333,0.454545,0.166667,0.166667,0.00,0.1,0.071429,0.250,0.142857,0.250,0.230769,0.000,0.555556,0.181818
CR,0.1875,0.090909,0.000000,0.000000,0.285714,0.500000,0.272727,0.166667,0.000000,0.00,0.0,0.000000,0.125,0.142857,0.000,0.307692,0.000,0.222222,0.090909
CZ,0.3125,0.090909,0.090909,0.285714,0.285714,0.500000,0.272727,0.333333,0.000000,0.00,0.1,0.000000,0.125,0.142857,0.125,0.307692,0.000,0.333333,0.181818
DE,0.3125,0.181818,0.181818,0.000000,0.285714,0.500000,0.181818,0.000000,0.166667,0.00,0.4,0.071429,0.125,0.142857,0.250,0.307692,0.125,0.444444,0.090909
DK,0.2500,0.090909,0.000000,0.000000,0.285714,0.500000,0.181818,0.000000,0.000000,0.00,0.0,0.000000,0.000,0.142857,0.000,0.307692,0.000,0.444444,0.090909
ES,0.1875,0.090909,0.000000,0.000000,0.285714,0.500000,0.272727,0.000000,0.166667,0.00,0.1,0.142857,0.125,0.142857,0.125,0.307692,0.000,0.333333,0.090909
FR,0.3125,0.090909,0.000000,0.142857,0.285714,0.500000,0.181818,0.000000,0.000000,0.00,0.0,0.000000,0.125,0.142857,0.000,0.384615,0.000,0.444444,0.090909


## PLOT CENSORSHIP

#### games category

In [117]:
df = df_unbiased[ df_unbiased['subcat']=='games' ]
game_censorship = pd.DataFrame( df.groupby(['case', 'domain'])['country'].unique() )
game_censorship['count'] = game_censorship['country'].apply( lambda x: len(x) )
game_censorship['country'] = game_censorship['country'].apply( lambda x: np.sort(x) )

In [142]:
game_censorship

country  \
case domain                                                                   
2    battle.net           [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     mmo-champion.com                                                  [CN]   
     pcgamer.com          [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     williamhill.com      [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
3    curse.com            [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     mmo-champion.com     [BR, CA, CR, CZ, DE, DK, ES, FR, GB, IN, IT, J...   
     planetminecraft.com  [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   

                          count  
case domain                      
2    battle.net              24  
     mmo-champion.com         1  
     pcgamer.com             24  
     williamhill.com         24  
3    curse.com               24  
     mmo-champion.com        23  
     planetminecraft.com     24

In [ ]:
game_censorship.to_html(RESULTS+subcat+'_censorship.html')

### Every subcat


In [159]:
print df_unbiased['subcat'].unique()

['vpn' 'news' 'adult' 'recreation' 'reference' 'computers' 'world' 'health'
 'arts' 'business' 'circum' 'sports' 'science' 'shopping' 'kids_and_teens'
 'home' 'games' 'regional' 'society']


In [160]:
for subcat in df_unbiased['subcat'].unique():
    df = df_unbiased[ df_unbiased['subcat'] == subcat ]
    censorship = pd.DataFrame( df.groupby(['case', 'domain'])['country'].unique() )
    censorship['count'] = censorship['country'].apply( lambda x: len(x) )
    censorship['country'] = censorship['country'].apply( lambda x: np.sort(x) )
    censorship.to_html(RESULTS + 'subcat_censorship_'+subcat+'.html')

In [161]:
censorship

country  \
case domain                                                                 
1    legacy.com                                              [CA, IT, RU]   
     slate.com                           [FR, NL, NO, PH, PL, SE, UA, US]   
2    collegeboard.org   [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     dailykos.com       [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     digg.com           [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     hollywoodlife.com                               [CA, IN, NL, PH, US]   
     legacy.com         [BR, CN, CR, CZ, DE, DK, ES, FR, GB, IN, JP, K...   
     menshealth.com     [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     nih.gov            [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     realsimple.com                  [CA, CN, IN, JP, KR, MX, MY, PH, US]   
     salon.com          [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     slate.com          [BR, CA, CN, CR, CZ, DE, DK, ES, GB, IN, IT, J...   
     snopes.com         [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
3    change.org         [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     dailykos.com                                                    [US]   
     digg.com                                                        [US]   
     goodsearch.com     [BR, CA, CN, CR, CZ, DE, DK, ES, FR, GB, IN, I...   
     hollywoodlife.com  [BR, CN, CR, CZ, DE, DK, ES, FR, GB, IT, JP, K...   
     nih.gov                                                         [NO]   
     realsimple.com     [BR, CR, CZ, DE, DK, ES, FR, GB, IT, NL, NO, P...   

                        count  
case domain                    
1    legacy.com             3  
     slate.com              8  
2    collegeboard.org      24  
     dailykos.com          23  
     digg.com              23  
     hollywoodlife.com      5  
     legacy.com            21  
     menshealth.com        24  
     nih.gov               23  
     realsimple.com         9  
     salon.com             24  
     slate.com             16  
     snopes.com            24  
3    change.org            24  
     dailykos.com           1  
     digg.com               1  
     goodsearch.com        24  
     hollywoodlife.com     19  
     nih.gov                1  
     realsimple.com        15

# WORLD MAP

- ignore the subcategories
- plot TOTAL censorship for every country

In [211]:
df = pd.DataFrame(df_unbiased.groupby(['country', 'case'])['sIP'].count()).unstack().reset_index().set_index('country')['sIP']
df['total'] = df.sum(axis=1)
df['case1'] = df[1]/df['total']
df['case2'] = df[2]/df['total']
df['case3'] = df[3]/df['total']
case1 = df['case1'].to_dict()
case2 = df['case2'].to_dict()
case3 = df['case3'].to_dict()

In [212]:
df

case,1,2,3,total,case1,case2,case3
country,,,,,,,
BR,7,151,22,180,0.038889,0.838889,0.122222
CA,12,146,22,180,0.066667,0.811111,0.122222
CN,6,144,30,180,0.033333,0.800000,0.166667
CR,3,157,20,180,0.016667,0.872222,0.111111
CZ,6,147,27,180,0.033333,0.816667,0.150000
DE,10,143,27,180,0.055556,0.794444,0.150000
DK,1,157,22,180,0.005556,0.872222,0.122222
ES,5,152,23,180,0.027778,0.844444,0.127778
FR,3,153,24,180,0.016667,0.850000,0.133333


In [216]:
create_world_map(case1, "Server to Client Censorship")

0.00555555555556 0.0666666666667
http://chart.apis.google.com/chart?cht=map:fixed=-60,-20,80,-35&chs=600x400&chma=0,60,0,0&chld=AA|AA|AA|AA|AA|AA|DK|JP|FR|CR|PL|UA|RU|IT|IN|ES|KR|MX|SE|PH|CN|CZ|MY|BR|NL|GB|US|DE|NO|CA&chco=808080|808080|c6dbef|9ecae1|6baed6|3182bd|08519c|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef&chdl=No+Data|0.0-20.0%|20.0-40.0%|40.0-60.0%|60.0-80.0%|80.0-100.0%&chdls=000000,14&chf=bg,s,EFF3FF&chtt=Server+to+Client+Censorship&chts=000000,20,c


In [217]:
create_world_map(case3, "Client to Server Censorship")

0.1 0.166666666667
http://chart.apis.google.com/chart?cht=map:fixed=-60,-20,80,-35&chs=600x400&chma=0,60,0,0&chld=AA|AA|AA|AA|AA|AA|PH|MY|NL|IN|CR|DK|JP|BR|NO|CA|KR|MX|PL|ES|FR|IT|GB|US|SE|DE|RU|CZ|UA|CN&chco=808080|808080|c6dbef|9ecae1|6baed6|3182bd|08519c|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef&chdl=No+Data|0.0-20.0%|20.0-40.0%|40.0-60.0%|60.0-80.0%|80.0-100.0%&chdls=000000,14&chf=bg,s,EFF3FF&chtt=Client+to+Server+Censorship&chts=000000,20,c


In [218]:
create_world_map(case2, "No Censorship")

0.794444444444 0.872222222222
http://chart.apis.google.com/chart?cht=map:fixed=-60,-20,80,-35&chs=600x400&chma=0,60,0,0&chld=AA|AA|AA|AA|AA|AA|DE|CN|CA|US|CZ|GB|RU|NO|UA|SE|BR|IT|ES|FR|PL|KR|MX|NL|IN|JP|PH|MY|DK|CR&chco=808080|808080|c6dbef|9ecae1|6baed6|3182bd|08519c|3182bd|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c|08519c&chdl=No+Data|0.0-20.0%|20.0-40.0%|40.0-60.0%|60.0-80.0%|80.0-100.0%&chdls=000000,14&chf=bg,s,EFF3FF&chtt=No+Censorship&chts=000000,20,c


In [167]:
def create_world_map(data, title, normalize=False, parts=[]):
    """Create a URL to a colorized world map with the google maps API

    Params:
    normalize- scale the input so that all the fractions for plotting
        fall between 0 and 1. This is useful if your input is not a
        fraction between 0 and 1

    Note: you need to open this in a browser to plot

    Note: we expect the data as dictionary where the keys are two
    letter country codes and the values are numbers that we should
    plot with the heat map

    """
    # convert the dict to a list of tuples with country code and value
    outputs = data.items()
    # sort by value
    outputs = sorted(outputs, key=lambda entry: entry[1])
    min_v, max_v = float(outputs[0][1]), float(outputs[-1][1])
    print min_v, max_v
    if not normalize:
        min_v, max_v = 0., 1.
    if parts != []:
        min_v, max_v = parts

    # partition the space depending on the number of colors
    buckets = np.array([int(float(x[1] - min_v)/max_v * 5) for x in outputs])
    outputs = zip(buckets, *zip(*outputs))
    # parts = np.linspace(min_v * 100., (max_v - min_v) * 100. / max_v, 5)
    parts = np.linspace(min_v * 100., max_v * 100., num=6)
    labels = []
    for index in range(1, 6):
        # num1 = int(parts[index - 1])
        # num2 = int(parts[index])
        labels.append("{:.1f}-{:.1f}%".format(parts[index - 1],
                                              parts[index]))

    # define our color map (10 colors at most)
    # colors = ["fff7ec", "fee8c8", "fdd49e", "fdbb84", "fc8d59",
    #           "ef6548", "d7301f", "b30000", "7f0000", "000000"]
    #
    # use this map because it is colorblind friendly and photocopy
    # safe (it should still print in grayscale)
    # colors = ["ffffcc", "a1dab4", "41b6c4", "2c7fb8", "253494"]
    # this colorscheme should also print in grayscale
    # colors = ["fef0d9", "fdcc8a", "fc8d59", "d7301f"]
    # colors = ["c6dbef", "9ecae1", "6baed6", "3182bd", "08519c", "eff3ff"]
    colors = ["c6dbef", "9ecae1", "6baed6", "3182bd", "08519c"]

    # put all the country arguments together
    map_args = ["http://chart.apis.google.com/chart?cht=map:"
                "fixed=-60,-20,80,-35", "chs=600x400", "chma=0,60,0,0"]
    # set all the unselected countries to be gray
    cnts, clrs = ["AA"], ["808080", "808080"]
    # setup the colors for the legend
    for color in colors:
        cnts.append("AA")
        clrs.append(color)
    # add the countries
    for (color, country, val) in outputs:
        cnts.append(country.upper())
        if color >= len(colors):
            color = len(colors) - 1
        clrs.append(colors[color])
    map_args.append("chld=" + "|".join(cnts))
    map_args.append("chco=" + "|".join(clrs))

    # add the legend
    map_args.append("chdl=No+Data|" + "|".join(labels))
    map_args.append("chdls=000000,14")
    # add the background to make the graph more visible
    map_args.append("chf=bg,s,EFF3FF")

    # create the title
    title = title.replace(" ", "+")
    map_args.append("chtt=" + title)
    map_args.append("chts=000000,20,c")

    print "&".join(map_args)


In [ ]:
import argparse
from base64 import b64decode
import collections
import csv
import dns.message
import dns.flags
import dns.rcode
import GeoIP
import geoip2.database
import json
from math import fabs
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pickle
import re
import scipy.stats as stats
import sqlite3
import traceback


# local imports
import interference
import interferenceAnalysis
import util


# SQL queries
# forwarderData table- get timing from the open resolvers to cdn nodes
insert_forwarder = ("INSERT INTO forwarderData (forwarderIP, "
                    "resolverIP, resolverIPTime, timestamp, forwarderCC, "
                    "resolverCC) VALUES "
                    "(?, ?, ?, ?, ?, ?)")

# domainData table- determining if the responses for certain sites are
# manipulated
insert_domain = ("INSERT INTO domainData (forwarderIP, domain, responseTime, "
                 "ips, servers, fullResp, isDup, hadParseIssue, status, "
                 "timestamp, rawID, ids) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, "
                 "?, ?, ?)")


def create_heat_map(data, x_labels, y_labels, title, filename, size=(8, 22),
                    font=None):
    """
    Plot a heatmap
    data = 
    """
    fig, ax = plt.subplots()
    cmap = plt.cm.hot
    cmap.set_bad(color='grey')
    ax.pcolormesh(data, cmap=cmap)

    # replace kids and teens with just kids if it is in the labels
    for index in range(len(x_labels)):
        if x_labels[index] == "kids_and_teens":
            x_labels[index] = "kids"

    ax.set_frame_on(False)
    fig = plt.gcf()
    fig.set_size_inches(*size)

    # now setup the axes
    ax.invert_yaxis()
    # ax.xaxis.tick_top()
    # ax.set_xticklabels(x_labels, minor=False)
    # ax.set_yticklabels(y_labels, minor=False)

    # put the major ticks at the middle of each cell
    ax.set_xticks(np.arange(data.shape[1])+0.5, minor=False)
    ax.set_yticks(np.arange(data.shape[0])+0.5, minor=False)
    if font is not None:
        ax.set_xticklabels(x_labels, minor=False, fontsize=font)
    else:
        ax.set_xticklabels(x_labels, minor=False)
    ax.set_yticklabels(y_labels, minor=False)
    plt.xticks(rotation=45, ha='right')

    # want a more natural, table-like display
    # ax.invert_yaxis()
    # x.xaxis.tick_top()

    # ax.xticks

    # setup a legend for the graph
    # cbar = plt.colorbar(cmap)
    # cbar.ax.get_yaxis().set_ticks([0.5, 2.5])
    # cbar.ax.set_yticklabels(["more censorship", "less censorship"])
    # cbar.ax.set_ylabel('Fraction of domains censored', rotation=270)

    # setup the legend for the graph by creating a 10x100 pixel colormap
    # fig = plt.gcf()
    # fig.set_size_inches(8, 26)
    # ax1 = plt.subplot(1, 5, 1)
    # ax1.set_frame_on(False)
    # cmap = plt.cm.hot
    # cmap.set_bad(color='grey')
    # ax1.pcolormesh(data, cmap=cmap)
    # ax1.set_xticks(np.arange(data.shape[1])+0.5, minor=False)
    # ax1.set_yticks(np.arange(data.shape[0])+0.5, minor=False)
    # ax1.invert_yaxis()
    # ax1.set_xticklabels(x_labels, minor=False, rotation=45, ha='right')
    # ax1.set_yticklabels(y_labels, minor=False)
    # ax1.xticks(rotation=45, ha='right')

    # ax2 = plt.subplot(1, 5, 5)
    # increments = np.outer(np.arange(0, 1, 0.01), np.ones(10))
    # ax2.axis("off")
    # ax2.imshow(increments, aspect='auto', cmap=cmap, origin='lower')
    # # setup a legend for the graph
    # ax2.get_yaxis().set_ticks([0.5, 2.5])
    # ax2.set_yticklabels(["more censorship", "less censorship"])
    # ax2.set_ylabel('Fraction of domains censored', rotation=270)

    plt.title(title)
    plt.grid()
    plt.savefig(filename, fmt='pdf')


def create_pie_chart(data, title, filename, cutoff=0.01, verbose=False):
    """Create a pie chart from the given data

    Params:
    data- a collections.Counter where the keys should be the labels to
        the graph

    """
    total = float(sum(data.values()))
    frac_labels = []
    other_count = 0
    # compute the fraction for each label
    for cnt in data.keys():
        fraction = float(data[cnt]) / total
        if verbose:
            print "{}: {}".format(cnt, fraction)
        # if this slice is too small, just add it to other small
        # totals and display them together
        if fraction < cutoff:
            other_count += data[cnt]
            continue
        frac_labels.append((fraction, cnt))
    if other_count > 0:
        frac_labels.append((float(other_count) / total, "Other"))
    frac_labels.sort(key=lambda entry: entry[1])
    fractions, labels = zip(*frac_labels)
    plt.figure()
    plt.pie(fractions, labels=labels, autopct='%1.1f%%',
            colors=('b', 'g', 'r', 'c', 'm', 'y', 'k', 'w'))
    plt.title(title)
    plt.savefig(filename, fmt='pdf')


def create_cdf(data, x_label, y_label, title, filename, log=False,
               invert=False):
    """Given a set of data, create a CDF of the data, optionally in log
    format

    """
    data.sort()
    # create the y data for the CDF
    if invert:
        y_data = [float(x) / float(len(data)) for x in range(len(data), 0, -1)]
    else:
        y_data = [float(x) / float(len(data)) for x in range(len(data))]
    fig, ax = plt.subplots()
    plt.grid()
    plt.plot(data, y_data)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    if invert:
        ax.invert_xaxis()

    if log:
        plt.xscale('log')

    plt.title(title)
    plt.savefig(filename, fmt='pdf')


def update_good_data_for_raw_censorship(good_values, checkpoint):
    """Update the checkpoint file with the correct good data"""

    with open(good_values, 'r') as file_p:
        cdn_data = pickle.load(file_p)
        cdn_sites = cdn_data['cdnSites']

    with open(checkpoint, 'r') as file_p:
        stage, data = pickle.load(file_p)

    data1 = interference.get_http_responses(cdn_sites.keys())
    data2 = interference.get_http_responses(cdn_sites.keys())
    good_data = {}
    for site in data1.keys():
        good_data[site] = interferenceAnalysis.check_http_header_consistency(data1[site]['headers'], data2[site]['headers'])
        good_data[site].update(data1[site])

    data['good_data'] = good_data
    with open(checkpoint, 'w') as file_p:
        pickle.dump([stage, data], file_p)


def parse_args():
    parser = argparse.ArgumentParser(prog='analyzeDNS')
    subparsers = parser.add_subparsers(dest='cmd',
                                       help='Select a subcommand to run')

    # add the command line arguments for importing data
    import_help = ("Import the results files into a SQL db for easier "
                   "querying")
    imp = subparsers.add_parser('import', help=import_help)
    imp.add_argument('--domain', '-d', help="domainData.data file",
                     required=True)
    imp.add_argument('--forwarder', '-f', help="forwarderData.data file",
                     required=True)
    imp.add_argument('--raw', '-r', help="rawResp.data file",
                     required=True)
    imp.add_argument('--interest', '-i', help="file for interesting domains",
                     required=True)
    imp.add_argument('--db', '-b', help="output sqlite db file",
                     required=True)
    imp.add_argument('--dups', '-s', help="output duplicates to this file",
                     default=None)

    # add the command line arguments for finding duplicate responses
    dup_help = ("Find the duplicate responses with different answers")
    dup = subparsers.add_parser('dup', help=dup_help)
    dup.add_argument('--dup', '-d', help="duplicates.data file", required=True)
    dup.add_argument('--raw', '-r', help="rawResp.data file", required=True)
    int_help = "file to output complete duplicates"
    dup.add_argument('--output', '-o', help=int_help, required=True)

    # add the command line arguments for exploring duplicate responses
    dup_help = ("Explore the duplicate responses")
    dup = subparsers.add_parser('dup-ex', help=dup_help)
    dup.add_argument('--raw', '-r', help="duplicates.data file", required=True)
    int_help = "file for outputing interesting domains"
    dup.add_argument('--interest', '-i', help=int_help, required=True)
    control_help = "known good values for the resolved IPs and their ASes"
    dup.add_argument('--control', '-c', help=control_help,
                     default="../data/control-ases-ips-27-jan-2015.txt")

    # add the command line argument for characterizing the dataset
    char_help = ("Characterize the dataset")
    char = subparsers.add_parser('char', help=char_help)
    char.add_argument("--basename", '-b', required=True,
                      help="basename for output files")
    muts = char.add_mutually_exclusive_group()
    muts.add_argument("--db", '-d', help="*.db file with domain info")
    graph_help = "use the given data file to graph without recomputing"
    muts.add_argument("--graph-with", '-g', help=graph_help)

    # add the CLI for adding info to a censorship file
    info_help = ("Add more information to a file of censored domains")
    info = subparsers.add_parser('add-info', help=info_help)
    info.add_argument("--input", '-i', required=True,
                      help="file of just forwarders and censored domains")
    info.add_argument("--output", '-o', required=True,
                      help="file to output new data to")
    info.add_argument("--categories", '-c', help="file to get categories from",
                      default="../../utility/sites/domain-to-cat-26-jan-2015.txt")
    info.add_argument("--forwarders", '-f', help="forwarderData.data file",
                      default="../data/forwarderData.data")

    # add the CLI for creating the server fingerprints file
    ip_help = ("Create server fingerprinting analysis file")
    ip_fpr = subparsers.add_parser('ip-fpr', help=ip_help)
    ip_fpr.add_argument("--dns-db", '-d', required=True,
                        help="censorship-scan.db file with DNS scan info")
    ip_fpr.add_argument("--good-db", '-g', required=True,
                        help="db with known good HTTP headers")
    ip_fpr.add_argument("--http-db", required=True,
                        help="db with http scanning results")
    ip_fpr.add_argument("--output", '-o', required=True,
                        help="File to output fingerprints to as a CSV file")
    control_help = "known good values for the resolved IPs and their ASes"
    ip_fpr.add_argument('--control', '-c', help=control_help,
                        default="../data/control-ases-ips-27-jan-2015.txt")

    # add the CLI for creating the NXDOMAIN analysis file for what
    # resolvers do monetization
    nxdom = ("Create NXDOMAIN analysis file to show which resolvers are "
             "doing NXDOMAIN error monetization")
    nxdom = subparsers.add_parser('nxdomain', help=ip_help)
    raw_help = ("rawResp.data file from the www.cs.princeton.edu NXDOMAIN "
                "control run")
    nxdom.add_argument("--raw", '-r', required=True, help=raw_help)
    nxdom.add_argument("--output", '-o', required=True,
                       help="output file for monetization servers")

    # add the CLI for doing the summary analysis
    summary = ("Create summary analysis file saying whether or not each "
               "response is censored")
    summ = subparsers.add_parser('summary', help=summary)
    summ.add_argument('--raw', help="rawResp.data file",
                      required=True)
    summ.add_argument("--dns-db", required=True,
                      help="censorship-scan.db file with DNS scan info")
    # summ.add_argument("--good-db", required=True,
    #                   help="db with known good HTTP headers")
    summ.add_argument("--http-db", required=True,
                      help="db with http scanning results")
    summ.add_argument("--output", required=True,
                      help="File to output summarized results")
    summ.add_argument("--ptr", required=True,
                      help="PTR lookups for resolved IPs")
    summ.add_argument("--categories", help="file to get categories from",
                      required=True,
                      default="../../utility/sites/domain-to-cat-26-jan-2015.txt")
    control_help = "known good values for the resolved IPs and their ASes"
    summ.add_argument('--control', help=control_help, required=True,
                      default="../data/control-ases-ips-27-jan-2015.txt")
    mon_help = "Monetization control analysis results"
    summ.add_argument('--mon-control', help=mon_help, required=True,
                      default="../data/monetization-control-output.txt")

    # add the CLI for graphing the summary analysis
    graphs = ("Graph the data from the summary analysis")
    graph = subparsers.add_parser('graph', help=graphs)
    graph.add_argument('--summary', help="summary file of what is censored",
                       required=True)
    graph.add_argument('--basename', help="basename for outputting graphs",
                       required=True)

    update = subparsers.add_parser('update',
                                   help="update good http server data")
    good_help = "known good values for the resolved IPs and their ASes"
    update.add_argument('--good', '-g', required=True, help=good_help)
    update.add_argument('--checkpoint', '-c',
                        help="checkpointing file to update",
                        default='summary-analysis-checkpoint')

    return parser.parse_args()


if __name__ == "__main__":
    args = parse_args()

    if args.cmd == "import":
        import_data(args.domain, args.forwarder, args.raw, args.interest,
                    args.db, duplicates=args.dups)
    elif args.cmd == "dup":
        find_duplicates(args.dup, args.raw, args.output)
    elif args.cmd == "dup-ex":
        explore_duplicates(args.raw, args.interest, args.control)
    elif args.cmd == "char":
        if args.db:
            characterize_data(args.db, args.basename)
        elif args.graph_with:
            plot_char_data(args.basename, in_file=args.graph_with)
        else:
            raise Exception("Invalid option. You must select either "
                            "--graph-with or --db")
    elif args.cmd == "add-info":
        add_info_to_censorship_file(args.input, args.output, args.categories,
                                    args.forwarders)
    elif args.cmd == "ip-fpr":
        create_ip_fpr_analysis_file(args.output, args.good_db, args.http_db,
                                    args.dns_db, args.control)
    elif args.cmd == "nxdomain":
        find_monetization(args.raw, args.output)
    elif args.cmd == "summary":
        # find_censorship_raw(args.raw, args.dns_db, args.http_db,
        #                     args.good_db, args.categories,
        #                     args.mon_control, args.control,
        #                     args.output, args.ptr)
        find_censorship_raw(args.raw, args.dns_db, args.http_db,
                            args.categories, args.mon_control,
                            args.control, args.output, args.ptr)
    elif args.cmd == "graph":
        create_graphs(args.summary, args.basename)
    elif args.cmd == "update":
        update_good_data_for_raw_censorship(args.good, args.checkpoint)
